In [1]:
### Gerekli Paketler
import pandas as pd
import numpy as np
import warnings
## Excel düzenlerken hata koduna ihtiyaç yok
warnings.filterwarnings('ignore')

In [3]:
a = pd.read_excel('./GSYH Panel Data IBBS1.xlsx')
## Boş Sütunları Birleştir
a['Yıl'] = a['Yıl'].ffill()
a['İl'] = a['İl'].ffill()
a['Tür'] = a['Tür'].ffill()
## Pivot Çakalım
a = a.pivot_table(index=['İl','Tür2'], columns='Yıl', values='Value').reset_index()

## Düzenle
k = list()
kk = list()
for zz in a['Tür2']:
    if 'Bin TL ve Cari Fiyatlarla ve ' in zz:
        k.append(zz.replace('Bin TL ve Cari Fiyatlarla ve ',''))
        kk.append('Cari')
    else:
        k.append(zz.replace('Bin TL ve Zincirlenmiş Hacim ve ',''))
        kk.append('Zincirlenmiş')
a['Tür'] = kk
a['Tür2'] = k

##GDP Hesapla
b = a[a['Tür2'] !=  'C. (İmalat Sanayi)'].groupby(['İl','Tür']).sum().reset_index()
b.columns = ['İl','Tür','2017_GDP','2022_GDP']

df = a.merge(b)

### Sektörleri Düzelt
df['Tür2'] = [zz.replace('A. (Tarım, Ormancılık Ve Balıkçılık)', 'Tarım').replace('BTE. (İmalat, Madencilik Ve Taş Ocakçılığı Ve Diğer Sanayiler)', 'Diğer Sanayi').replace('C. (İmalat Sanayi)', 'İmalat').replace('F. (İnşaat)', 'İnşaat').replace('GTI. (Toptan Ve Perakende Ticaret, Ulaştırma Ve Depolama, Konaklama Ve Yiyecek Hizmeti Faaliyetleri)', 'Hizmet').replace('J. (Bilgi Ve İletişim)', 'Hizmet').replace('K. (Finans Ve Sigorta Faaliyetleri)', 'Bankacılık').replace('L. (Gayrimenkul Faaliyetleri)', 'Hizmet').replace('M_N. (Mesleki, Bilimsel, Teknik, İdari Ve Destek Hizmet Faaliyetleri)', 'Hizmet').replace('OTQ. (Kamu Yönetimi Ve Savunma, Eğitim, İnsan Sağlığı Ve Sosyal Hizmet Faaliyetleri)', 'Kamu').replace('RTU. (Diğer Hizmetler)','Hizmet') for zz in df['Tür2']]

## Loopu kötü yazıyorum ama süreyi etkilemeyecek
## Amacımız İmalat Sanayi'ni (üretkeni) diğer sanayi'den (üretken olmayandan çıkartmak)
df2 = pd.DataFrame()
for tur in df['Tür'].unique():
    for il in df['İl'].unique():
        a = df[(df['İl'] == il) & (df['Tür'] == tur)].reset_index(drop=True)
        a[a['Tür2'] == 'İmalat']
        b = a[a['Tür2'] == 'Diğer Sanayi']
        a = a.drop(a[a['Tür2'] == 'Diğer Sanayi'].index[0])
        b[2017] = b[2017] - a[a['Tür2'] == 'İmalat'].reset_index()[2017][0]
        b[2022] = b[2022] - a[a['Tür2'] == 'İmalat'].reset_index()[2022][0]

        a = pd.concat([a,b])

        df2 = pd.concat([df2,a])

df = df2.groupby(['İl','Tür2','Tür','2017_GDP','2022_GDP']).sum().reset_index()

### Sectoral Shares
df['2017_s'] = df[2017]/df['2017_GDP']*100
df['2022_s'] = df[2022]/df['2022_GDP']*100

## Growth Rates
df['g_s'] = df[2022]/df[2017]
df['g_gdp'] = df['2022_GDP']/df['2017_GDP']

df.to_excel('GDP_final.xlsx', index=False)

FileNotFoundError: [Errno 2] No such file or directory: './GSYH Panel Data IBBS1.xlsx'